In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import Birch
import requests
import json

np.random.seed(0)

training_data = pd.read_csv('data_tr.txt',sep='\t',header=None)
actual_data=pd.read_csv('gene_names.txt',header=None)

test_data = pd.read_csv('data_ts.txt',sep = '\t', header = None)
labels = actual_data[0].tolist()

training_data.columns = labels
test_data.columns = labels

features_toDrop =pd.read_csv('ColumnsDropped.txt',header=None)
features_toDrop = features_toDrop[0].tolist()

training_data.drop(features_toDrop, axis = 1, inplace = True)
test_data.drop(features_toDrop, axis = 1, inplace = True)


scaler = MinMaxScaler()
scaler.fit(training_data)
scaled_features_train = scaler.transform(training_data)
scaled_features_test = scaler.transform(test_data)

training_data = scaled_features_train
test_data = scaled_features_test

pca = PCA(n_components=45,svd_solver='full')
pca.fit(training_data)
train_pca = pca.transform(training_data)
test_pca = pca.transform(test_data)

from sklearn.cluster import Birch

# Creating the BIRCH clustering model
model = Birch(n_clusters = 16)
  
# Fit the data (Training)
model.fit(train_pca)
  
# Predict the same data
predictions = model.predict(test_pca)


url = "https://www.csci555competition.online/scoretest"
payload = json.dumps(predictions.tolist())
    
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

ConnectionError: HTTPSConnectionPool(host='www.csci555competition.online', port=443): Max retries exceeded with url: /scoretest (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000022B5586F550>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))